In [1]:
!pip install --upgrade pactools
!pip install hyperas
!pip install hyperopt
!pip install -U -q PyDrive

Requirement already up-to-date: pactools in /usr/local/lib/python3.6/dist-packages (0.3)


In [2]:
# Install the PyDrive wrapper & import libraries.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='hyperas_resnet.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('hyperas_resnet.ipynb')

In [3]:
from hyperopt import Trials, STATUS_OK, tpe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical

#CROSS VALIDATION
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from pactools.grid_search import GridSearchCVProgressBar

#WRAPPERS NEEDED FOR GRIDSEARCH
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split

from hyperas import optim
from hyperas.distributions import choice, uniform

In [4]:
def data():
  train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
  train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)
  train2,test1, train_labels2,test_labels1 = train_test_split(train1, train_labels1, test_size=0.2,random_state=42)
  x_train=train2/225.0
  y_train = pd.get_dummies(train_labels2)

  x_test=test1/225.0
  y_test = pd.get_dummies(test_labels1)
  return x_train,y_train,x_test,y_test

In [5]:
IMG_SHAPE1=(64,64,3)

resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

In [6]:
def create_model_dense_resnet(x_train,y_train,x_test,y_test): 
  IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

  resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')
  tempre=resnet
  froz={{choice([3,6,10,15,20,25,30,40,60])}}
  for layer in tempre.layers[:(-1)*froz]:
    layer.trainable = False
  
  model = tf.keras.Sequential()
  model.add(tempre)
  if ({{choice([0,1])}}):
    model.add(keras.layers.Flatten())
  else:
    model.add(keras.layers.GlobalAveragePooling2D())
  dense_layer_size={{choice([0,1,2,3])}}
  if (dense_layer_size==3):
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout({{choice([0.0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65])}}))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout({{choice([0.0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65])}}))
  elif (dense_layer_size==2):
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout({{choice([0.0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65])}}))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout({{choice([0.0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65])}}))
  elif (dense_layer_size==1):
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout({{choice([0.0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65])}}))
  else:
    layers=[]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout(0.1))
  model.add(keras.layers.Dense(3,activation="softmax"))
  print('froz=',froz,'dense_layers=',dense_layer_size)
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
  result = model.fit(x_train, y_train,
              batch_size=1000,
              epochs=5,
              verbose=2,
              validation_data=(x_test, y_test))
  validation_acc = np.amax(result.history['val_accuracy']) 
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


In [ ]:
best_run, best_model = optim.minimize(model=create_model_dense_resnet,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=300,
                                          notebook_name='hyperas_resnet',
                                          trials=Trials())
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    from pydrive.auth import GoogleAuth
except:
    pass

try:
    from pydrive.drive import GoogleDrive
except:
    pass

try:
    from google.colab import auth
except:
    pass

try:
    from oauth2client.client import GoogleCredentials
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from matplotlib.ticker import MaxNLocator
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras import datasets, layers, models
except:
    pass

try:
    from keras.preprocessing.text import Tokenizer
except:
    pass

try:
    from keras.models import Sequential, Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling